<a href="https://colab.research.google.com/github/ethandavenport/Optimization-I-Project-2/blob/main/linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

In [3]:
url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/main/roi_company1.csv"
df_roi = pd.read_csv(url)

In [4]:
df_roi.head()

,Platform,Tier,LowerBound,UpperBound,ROI
0,Print,1,0.0,0.7,0.0345
1,Print,2,0.7,2.7,0.0305
2,Print,3,2.7,4.7,0.0276
3,Print,4,4.7,inf,0.0234
4,TV,1,0.0,4.3,0.0608


In [5]:
# Amounts are in MILLIONS of dollars (so budget=10 means $10M)
# --- Parameters (change if needed) ---
budget = 10.0        # total budget in millions
platform_cap = 3.0   # per-platform max in millions
bigM = budget        # big-M for "infinite" tiers (safe because total budget <= budget)

# Compute tier widths (maximum amount that can be placed in that tier)
def tier_width(row):
    if np.isfinite(row['UpperBound']):
        return max(0.0, row['UpperBound'] - row['LowerBound'])
    else:
        return bigM  # allow up to the whole budget for an "infinite" tier; platform cap will restrict
df_roi['Width'] = df_roi.apply(tier_width, axis=1)

# Build model
m = gp.Model()

Restricted license - for non-production use only - expires 2026-11-23


In [6]:
# Decision vars: amount invested in each tier (millions)
vars_by_index = {}
for idx, row in df_roi.iterrows():
    name = f"x_{int(idx)}_{row['Platform']}_t{int(row['Tier'])}"
    ub = row['Width']
    # some widths could be zero if Upper==Lower; skip creating vars that can't take anything
    if ub <= 0:
        v = m.addVar(lb=0.0, ub=0.0, name=name)
    else:
        v = m.addVar(lb=0.0, ub=ub, name=name)
    vars_by_index[idx] = v

In [7]:
# Objective: maximize total return = sum(ROI * invested_in_tier)
obj_expr = gp.quicksum(df_roi.loc[idx, 'ROI'] * vars_by_index[idx] for idx in vars_by_index)
m.setObjective(obj_expr, GRB.MAXIMIZE)

In [8]:
# 1) Total budget constraint
m.addConstr(gp.quicksum(vars_by_index[idx] for idx in vars_by_index) <= budget)

# Helper: platform -> list of tier indices
platform_to_indices = {}
for idx, row in df_roi.iterrows():
    platform_to_indices.setdefault(row['Platform'], []).append(idx)

# 2) Per-platform cap: total invested in each platform <= platform_cap
for platform, idx_list in platform_to_indices.items():
    m.addConstr(gp.quicksum(vars_by_index[idx] for idx in idx_list) <= platform_cap)

# Business constraints:
# a) Print + TV <= Facebook + Email
def platform_total(platform):
    idxs = platform_to_indices.get(platform, [])
    return gp.quicksum(vars_by_index[i] for i in idxs)

m.addConstr(platform_total('Print') + platform_total('TV') <= platform_total('Facebook') + platform_total('Email'))

# b) Social media (Facebook, LinkedIn, Instagram, Snapchat, Twitter) >= 2 * (SEO + AdWords)
social_platforms = ['Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter']
social_sum = gp.quicksum(platform_total(p) for p in social_platforms)
m.addConstr(social_sum >= 2.0 * (platform_total('SEO') + platform_total('AdWords')))

<gurobi.Constr *Awaiting Model Update*>

In [9]:
# Solve
m.Params.OutputFlag = 0 # tell gurobi to shut up!!
m.optimize()

In [10]:
# Print results
if m.status == GRB.OPTIMAL or m.status == GRB.TIME_LIMIT or m.status == GRB.SUBOPTIMAL:
    print('\n=== Optimal allocation (amounts in MILLIONS) ===')
    # Per-platform totals
    platform_alloc = {}
    for platform, idxs in platform_to_indices.items():
        total = sum(vars_by_index[i].X for i in idxs)
        platform_alloc[platform] = total
        print(f"{platform:10s}: {total:.4f} (per-tier breakdown below)")
        for i in idxs:
            invested = vars_by_index[i].X
            if invested > 1e-8:
                row = df_roi.loc[i]
                print(f"   tier {int(row['Tier'])}: {invested:.4f}  (ROI={row['ROI']:.4f}, width={row['Width']:.2f})")
    total_invested = sum(v.X for v in vars_by_index.values())
    total_return = sum(df_roi.loc[i, 'ROI'] * vars_by_index[i].X for i in vars_by_index)
    print(f"\nTotal invested: {total_invested:.4f} / {budget:.4f} (M)")
    print(f"Estimated total return: {total_return:.6f} (in MILLIONS of dollars returned)")
    # If you want dollar-level return, multiply by 1e6
else:
    print("Model did not solve to optimality. Status:", m.status)


=== Optimal allocation (amounts in MILLIONS) ===
Print     : 0.0000 (per-tier breakdown below)
TV        : 3.0000 (per-tier breakdown below)
   tier 1: 3.0000  (ROI=0.0608, width=4.30)
SEO       : 0.0000 (per-tier breakdown below)
AdWords   : 1.0000 (per-tier breakdown below)
   tier 1: 1.0000  (ROI=0.0419, width=2.00)
Facebook  : 0.0000 (per-tier breakdown below)
LinkedIn  : 0.0000 (per-tier breakdown below)
Instagram : 3.0000 (per-tier breakdown below)
   tier 1: 2.9000  (ROI=0.0574, width=2.90)
   tier 2: 0.1000  (ROI=0.0498, width=10.00)
Snapchat  : 0.0000 (per-tier breakdown below)
Twitter   : 0.0000 (per-tier breakdown below)
Email     : 3.0000 (per-tier breakdown below)
   tier 1: 3.0000  (ROI=0.0493, width=5.90)

Total invested: 10.0000 / 10.0000 (M)
Estimated total return: 0.543640 (in MILLIONS of dollars returned)
